In [ ]:
import catboost
import pandas as pd
from mlModules import control
import pickle

In [ ]:
def predict(days, model, metadata):
    results = pd.DataFrame()
    tomorrow = pd.to_datetime("today") + pd.DateOffset(days=1)
    for day in pd.date_range(start = tomorrow, periods = days, normalize = True):
        x = metadata.copy()
        x['DAY'] = day
        x['WEEKNUMBER'] = day.week
        x['WEEKDAY'] = day.weekday()
        x['MONTH'] = day.month
        x['PREDICTEDLEADTIMES'] = model.predict(x).clip(min=0)
        results = results.append(x, ignore_index=True)
    resBlob = pickle.dumps(results)
    api.send("prediction", resBlob)

In [ ]:
def onInput(days, metadataBlob, modelBlob):
    global model, predDays, metadata
    model = pickle.loads(modelBlob)
    metadata = pickle.loads(metadataBlob)
    predDays = int(days)
    print(model.is_fitted())
    predict(predDays, model, metadata)
api.set_port_callback(["days", "metadata", "model"], onInput)